In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import time
import pandas as pd


In [2]:
driver = webdriver.Chrome('/usr/local/bin/chromedriver')

/Users/zachariah/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [3]:
driver.get('https://www.indeed.com/')

In [4]:
text = driver.find_element("xpath", '//*[@id="text-input-what"]')

In [5]:
text.send_keys('Data Analyst')

In [6]:

text.send_keys(Keys.ENTER)

In [7]:
time.sleep(2)

In [8]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [9]:
postings = soup.find_all('div',class_ = 'job_seen_beacon')

In [10]:
len(postings)

15

In [13]:
df = pd.DataFrame({'Link':[''],'Job Title':[''],'Company':[''], 'Location':[''], 'Salary':[''],'Posted':['']})
        

In [14]:
while True:
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    postings = soup.find_all('div', class_ ='job_seen_beacon')
    for post in postings:
        link = post.find('a', class_ ='jcs-JobTitle css-jspxzf eu4oa1w0').get('href')
        link_full = 'https://www.indeed.com' + link
        try:
            jobtitle = post.find('h2', class_= 'jobTitle css-1h4a4n5 eu4oa1w0').text.strip()
        except:
            jobtitle = 'N/A'
        company = post.find('span', class_= 'companyName').text.strip()
        try:
            location = post.find('div', class_= 'companyLocation').text.strip()
        except:
            location = 'N/A'
        try:
            salary = post.find('div', class_ = 'attribute_snippet').text.strip()
        except:
            salary = 'N/A'
        try:
            dated = post.find('span', class_='date').text.strip()
            posted = dated.split(" ")[1]
        except:
            posted = 'N/A'
        df = df.append({'Link':link_full,'Job Title':jobtitle, 'Company':company,'Location':location, 'Salary':salary,'Posted':posted}, ignore_index = True)

    try:
        next_p = soup.find_all('a', {'aria-label':'Next'})[0].get("href")
        driver.get('https://www.indeed.com' +next_p)
    except:
        break


In [15]:
df

,Link,Job Title,Company,Location,Salary,Posted
0,,,,,,
1,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Data Analyst Apprenticeship at Merck Pharmaceu...,OpenClasssrooms,"Wilmington, DE",$22 - $25 an hour,10
2,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Investment Data Analyst,Vanguard,"Malvern, PA",N/A,16
3,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"Data Analyst - Diversity, Equity and Inclusion",Vanguard,"Malvern, PA",N/A,18
4,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Fraud Data Analyst,Continental Finance Company,"Hybrid remote in Wilmington, DE 19808","$50,000 - $65,000 a year",30+
...,...,...,...,...,...,...
228,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Senior Data Risk Analyst,Indeed,United States,"$75,000 - $109,000 a year",30+
229,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,N/A,International Mission Board,Remote in United States,Full-time,4
230,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Operational Data Analyst I,Computershare,Remote in United States,N/A,30+
231,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Junior Business Analyst(B.A.),Logical Paradigm LLC,United States,$27 an hour,1


In [47]:
#The code below just sorts the dataframe by posting date
df['Date_num'] = df['Posted'].apply(lambda x: x[:2].strip())

def integer(x):
    try:
        return int(x)
    except:
        return x
    
df['Date_num2'] = df['Date_num'].apply(integer)
df.sort_values(by = ['Date_num2','Salary'], inplace = True)  

df = df[['Link', 'Job Title','Company','Location', 'Salary','Posted']]



/Users/zachariah/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/zachariah/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/zachariah/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in th

In [48]:
#exports the dataframe as a csv
df.to_csv('/Users/zachariah/Desktop/indeed_jobs.csv')

In [49]:
df

,Link,Job Title,Company,Location,Salary,Posted
231,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Junior Business Analyst(B.A.),Logical Paradigm LLC,United States,$27 an hour,1
57,https://www.indeed.com/rc/clk?jk=8aa9d31026555...,N/A,"JPMorgan Chase Bank, N.A.","Wilmington, DE",Full-time,1
58,https://www.indeed.com/rc/clk?jk=23ba6ba476f96...,N/A,"JPMorgan Chase Bank, N.A.","Wilmington, DE",Full-time,1
122,https://www.indeed.com/rc/clk?jk=1e2394b0e9e5a...,N/A,BNP Paribas,"Chesterbrook, PA+1 location",Full-time,1
132,https://www.indeed.com/company/Cecil-College/j...,Senior Research and Data Visualization Analyst,Cecil College,"North East, MD 21901","$65,207 - $75,478 a year",2
...,...,...,...,...,...,...
19,https://www.indeed.com/rc/clk?jk=f7922a340c611...,N/A,"JPMorgan Chase Bank, N.A.","Newark, DE 19713",Full-time,N/A
126,"https://www.indeed.com/company/Viewstream,-Inc...",N/A,"Viewstream, Inc.","Wilmington, DE","$75,000 - $100,000 a year",ongoing
5,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,N/A,Volga Partners,Pennsylvania,$15 - $17 an hour,posted
84,https://www.indeed.com/rc/clk?jk=b8e0257ee4fac...,N/A,Lincoln Financial,"Remote in Radnor, PA","$39,900 - $83,700 a year",posted


In [67]:
#Code below sends an email to whomever through python
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email import encoders

#Input the email account that will send the email and who will receiving it
sender = 'webscrap85@gmail.com'
receiver = 'webscrap85@gmail.com'




In [68]:
#Creates the Message, Subject line, From and To
msg = MIMEMultipart()
msg['Subject'] = 'New Jobs on Indeed'
msg['From'] = sender
msg['To'] = ','.join(receiver)



In [69]:
#Adds a csv file as an attachment to the email (indeed_jobs.csv is our attahced csv in this case)
part = MIMEBase('application', 'octet-stream')
part.set_payload(open('/Users/zachariah/Desktop/indeed_jobs.csv', 'rb').read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', 'attachment; filename ="indeed_jobs.csv"')
msg.attach(part)



In [70]:
#Will login to your email and actually send the message above to the receiver
s = smtplib.SMTP_SSL(host = 'smtp.gmail.com', port = 465)
s.login(user = 'webscrap85@gmail.com', password = 'ycfd cqqe tzci uhos')
s.sendmail(sender, receiver, msg.as_string())
s.quit()

(221,
 b'2.0.0 closing connection fp33-20020a05622a50a100b0031eaea83fc1sm9722054qtb.47 - gsmtp')